In [1]:
import keras._tf_keras.keras as keras
from keras._tf_keras.keras.models import Model
from keras._tf_keras.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, GlobalAveragePooling3D
from keras._tf_keras.keras.layers import Dropout, concatenate, multiply, Dense, GroupNormalization
from keras._tf_keras.keras.layers import LeakyReLU

2025-01-03 23:15:48.018589: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-03 23:15:48.028820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735926348.039255  217855 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735926348.042062  217855 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 23:15:48.052369: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
class InstanceNormalization(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.norm = GroupNormalization(groups=-1)
    def call(self, x):
        return self.norm(x)

class SE_Layer(keras.layers.Layer):
    def __init__(self, ch, ratio = 16, **kwargs):
        super(SE_Layer, self).__init__(**kwargs)
        self.gl = GlobalAveragePooling3D(keepdims=True)
        self.fc1 = Dense(ch//ratio, activation='relu')
        self.fc2 = Dense(ch, activation='sigmoid')
    def call(self, input_block):
        x = self.gl(input_block)
        x = self.fc1(x)
        x = self.fc2(x)
        return multiply([input_block, x])


In [3]:
def My_LATUP(input_shape: tuple):
    inputs = Input(input_shape)

    # Encoder Block 1 (Parallel Convolutions Block) (E1)
    e1_pc_embed = Conv3D(32, (3, 3, 3), strides=(1, 1, 1), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E1_PC_Embedded_Layer')(inputs)
    e1_pc_conv1 = Conv3D(32, (1, 1, 1), strides=(1, 1, 1), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E1_PC_Conv1_Layer')(e1_pc_embed)
    e1_pc_conv2 = Conv3D(32, (3, 3, 3), strides=(1, 1, 1), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E1_PC_Conv2_Layer')(e1_pc_embed)
    e1_pc_conv3 = Conv3D(32, (5, 5, 5), strides=(1, 1, 1), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E1_PC_Conv3_Layer')(e1_pc_embed)

    e1_pc_maxpool1 = MaxPooling3D(pool_size=(2, 2, 2), name='E1_maxpool1_Layer')(e1_pc_conv1)
    e1_pc_maxpool2 = MaxPooling3D(pool_size=(2, 2, 2), name='E1_maxpool2_Layer')(e1_pc_conv2)
    e1_pc_maxpool3 = MaxPooling3D(pool_size=(2, 2, 2), name='E1_maxpool3_Layer')(e1_pc_conv3)

    e1_pc_concat = concatenate([e1_pc_maxpool1, e1_pc_maxpool2, e1_pc_maxpool3], name='E1_concat_Layer')

    #Encoder Block 2 (E2)
    e2_se1 = SE_Layer(96, ratio=8, name='E2_SE1_Layer')(e1_pc_concat)
    e2_conv1 = Conv3D(64, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E2_Conv1_Layer')(e2_se1)
    e2_instance = InstanceNormalization(name='E2_instance_Layer')(e2_conv1)
    e2_conv2 = Conv3D(64, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E2_Conv2_Layer')(e2_instance)
    e2_dropout = Dropout(0.2, name='E2_Drop')(e2_conv2)
    e2_maxpool1 = MaxPooling3D(pool_size=(2, 2, 2), name='E2_maxpool1_Layer')(e2_dropout)

    #Encoder Block 3 (E3)
    e3_se1 = SE_Layer(64, ratio=8, name='E3_SE1_Layer')(e2_maxpool1)
    e3_conv1 = Conv3D(128, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E3_Conv1_Layer')(e3_se1)
    e3_instance = InstanceNormalization(name='E3_instance_Layer')(e3_conv1)
    e3_conv2 = Conv3D(128, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='E3_Conv2_Layer')(e3_instance)
    e3_dropout = Dropout(0.2, name='E3_drop')(e3_conv2)
    e3_maxpool1 = MaxPooling3D(pool_size=(2, 2, 2), name='E3_maxpool1_Layer')(e3_dropout)

    bn_se1 = SE_Layer(128, ratio=8, name='BN_SE1_Layer')(e3_maxpool1)

    #Decoder Block 3 (D3)
    d3_up = UpSampling3D(size=(2, 2, 2), name='D3_up')(bn_se1)
    d3_conv1 = Conv3D(128, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D3_Conv1_Layer')(d3_up)
    d3_instance = InstanceNormalization(name='D3_instance_Layer')(d3_conv1)
    d3_concat = concatenate([d3_instance, e3_dropout], name='D3_concat_Layer')
    d3_conv2 = Conv3D(128, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D3_Conv2_Layer')(d3_concat)
    d3_se1 = SE_Layer(128, ratio=8, name='D3_SE1_Layer')(d3_conv2)
    d3_dropout = Dropout(0.2, name='D3_drop')(d3_se1)
    d3_conv3 = Conv3D(128, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D3_Conv3_Layer')(d3_dropout)

    #Decoder Block 2(D2)
    d2_up = UpSampling3D(size=(2, 2, 2), name='D2_up')(d3_conv3)
    d2_conv1 = Conv3D(64, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D2_Conv1_Layer')(d2_up)
    d2_instance = InstanceNormalization(name='D2_instance_Layer')(d2_conv1)
    d2_concat = concatenate([d2_instance, e2_dropout], name='D2_concat_Layer')
    d2_conv2 = Conv3D(64, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D2_Conv2_Layer')(d2_concat)
    d2_se1 = SE_Layer(64, ratio=8, name='D2_SE1_Layer')(d2_conv2)
    d2_dropout = Dropout(0.2, name='D2_drop')(d2_se1)
    d2_conv3 = Conv3D(64, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D2_Conv3_Layer')(d2_dropout)

    #Decoder Block 1(D1)
    d1_up = UpSampling3D(size=(2, 2, 2), name='D1_up')(d2_conv3)
    d1_conv1 = Conv3D(32, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D1_Conv1_Layer')(d1_up)
    d1_instance = InstanceNormalization(name='D1_instance_Layer')(d1_conv1)
    d1_concat = concatenate([d1_instance, e1_pc_embed], name='D1_concat_Layer')
    d1_conv2 = Conv3D(32, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D1_Conv2_Layer')(d1_concat)
    d1_se1 = SE_Layer(32, ratio=8, name='D1_SE1_Layer')(d1_conv2)
    d1_dropout = Dropout(0.2, name='D1_drop')(d1_se1)
    d1_conv3 = Conv3D(32, (3, 3, 3), activation=LeakyReLU(negative_slope=0.1), padding='same', name='D1_Conv3_Layer')(d1_dropout)

    #Probablity Filter
    prob = Conv3D(4, (1, 1, 1), activation='softmax', name='prob')(d1_conv3)

    model = Model(inputs=inputs, outputs=prob, name='MY_LATUP')
    return model


In [4]:
model = My_LATUP(input_shape = (128, 128, 128, 3))
model.summary(line_length=110)

I0000 00:00:1735926352.798262  217855 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "MY_LATUP"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape              ┃          Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)       │ (None, 128, 128, 128, 3)  │                0 │ -                          │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Embedded_Layer (Conv3D)  │ (None, 128, 128, 128, 32) │            2,624 │ input_layer[0][0]          │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv1_Layer (Conv3D)     │ (None, 128, 128, 128, 32) │            1,056 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv2_Layer (Conv3D)     │ (None, 128, 128, 128, 32) │           27,680 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_PC_Conv3_Layer (Conv3D)     │ (None, 128, 128, 128, 32) │          128,032 │ E1_PC_Embedded_Layer[0][0] │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool1_Layer              │ (None, 64, 64, 64, 32)    │                0 │ E1_PC_Conv1_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool2_Layer              │ (None, 64, 64, 64, 32)    │                0 │ E1_PC_Conv2_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_maxpool3_Layer              │ (None, 64, 64, 64, 32)    │                0 │ E1_PC_Conv3_Layer[0][0]    │
│ (MaxPooling3D)                 │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E1_concat_Layer (Concatenate)  │ (None, 64, 64, 64, 96)    │                0 │ E1_maxpool1_Layer[0][0],   │
│                                │                           │                  │ E1_maxpool2_Layer[0][0],   │
│                                │                           │                  │ E1_maxpool3_Layer[0][0]    │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_SE1_Layer (SE_Layer)        │ (None, 64, 64, 64, 96)    │            2,412 │ E1_concat_Layer[0][0]      │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Conv1_Layer (Conv3D)        │ (None, 64, 64, 64, 64)    │          165,952 │ E2_SE1_Layer[0][0]         │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_instance_Layer              │ (None, 64, 64, 64, 64)    │              128 │ E2_Conv1_Layer[0][0]       │
│ (InstanceNormalization)        │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Conv2_Layer (Conv3D)        │ (None, 64, 64, 64, 64)    │          110,656 │ E2_instance_Layer[0][0]    │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ E2_Drop (Dropout)              │ (None, 64, 64, 64, 64)    │                0 │ E2_Conv2_Layer[0][0]       │
├───

 Total params: 3,575,492 (13.64 MB)

 Trainable params: 3,575,492 (13.64 MB)

 Non-trainable params: 0 (0.00 B)